# Natural Language Processing (linguistische Datenverarbeitung)

## Lernziele
* Text so transformieren können, dass wir Methoden des maschinellen Lernens darauf anwenden können.
* Emotionen in Text klassifizieren können.
* Die Performance eines Klassifikators evaluieren können, der mehr als zwei Klassen klassifiziert.

## Text als Daten

Wir verwenden einen Datensatz mit englischen Sätzen, denen jeweils eine von sechs möglichen Emotionen (Traurigkeit, Freude, Liebe, Wut, Angst und Überraschung) zugeordnet ist. Der Datensatz stellt einen Auszug des [Emotions](https://www.kaggle.com/datasets/bhavikjikadara/emotions-dataset?resource=download) Datensatzes dar, indem jede Emotion gleich oft (mit 1000 Beispiele) repräsentiert ist. Wir möchten den Datensatz verwenden, um einen Klassifikator zu trainieren, der Emotionen in Text erkennen kann.

*Der Datensatz darf unter einer [CC BY 4.0 Lizenz](https://creativecommons.org/licenses/by/4.0/) verwendet, modifiziert und veröffentlicht werden.*

In [24]:
import pandas as pd
url = "https://drive.google.com/uc?id=1vfgHvGBMOAyozxlbwTGElPcyofeqjP9w"
emotionen = pd.read_csv(url)
emotionen.head(10)

,text,label,text_label
0,i have to admit despite all my optimism i stil...,4,fear
1,i feel helpless and weary,0,sadness
2,i have this terrible feeling the world knows s...,3,anger
3,i could feel him raise to look at me with the ...,2,love
4,i really got rejected in the end i think i wil...,0,sadness
5,i feel so amazed by women who can balance work...,5,surprise
6,i am just reading about your mom always being ...,4,fear
7,i have just been introduced to feels very strange,5,surprise
8,i or fresh energy anytime that you are in the ...,3,anger
9,i feel hateful emotions like irritation frustr...,3,anger


In [ ]:
# 0: sadness
# 1: joy
# 2: love
# 3: anger
# 4: fear
# 5: surprise


In [2]:
emotionen["text_label"].value_counts()

,count
text_label,
fear,1000
sadness,1000
anger,1000
love,1000
surprise,1000
joy,1000


In [3]:
# wir weisen unsere Beobachtungen (Sätze) und Zielwerte (Emotionen)
# Variablen zu:
x = emotionen["text"]
y = emotionen["label"]

Wie in der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO4_1_text_als_daten/) besprochen, müssen wir die Texte dafür erst in Zahlen transformieren - eine sogenannte [Einbettung](https://janalasser.at/lectures/MC_KI/VO4_1_text_als_daten/#/1/0/0) der Texte erstellen. Eine Möglichkeit dafür wäre, [one-hot encoding](https://janalasser.at/lectures/MC_KI/VO4_1_text_als_daten/#/1) anzuwenden und jedes Wort als eigene "Kategorie" zu verstehen.

<div>
<img src="https://drive.google.com/uc?id=1Z_uUj8zcwmlgTqKea65UC_D-y2vUn1Bi" width="600"/>
</div>

In [4]:
# in scikit-learn können wir das mit dem CountVectorizer umsetzen
from sklearn.feature_extraction.text import CountVectorizer

saetze = [
    "I love apples",
    "I received a cat",
    "tomorrow is Tuesday"]

count_vect = CountVectorizer()
count_vect.fit(saetze)

CountVectorizer()

In [5]:
# für das Beispiel mit nur drei Sätzen ist die Anzahl der Worte (features)
# noch gut handhabbar. Außerdem ignoriert der CountVectorizer Worte, die
# nur einen Buchstaben haben.
count_vect.get_feature_names_out()

array(['apples', 'cat', 'is', 'love', 'received', 'tomorrow', 'tuesday'],
      dtype=object)

Wie viele Dimensionen (features) hätte das DataFrame, den wir so aus dem `emotionen` Datensatz generieren würden? Um das herauszufinden, können wir die Anzahl der verschiedenen Worte zählen, die in den Sätzen im Datensatz enthalten sind und eine Länge > 1 haben:

In [7]:
# Liste zum Speichern der Worte
alle_worte = []

# iteriere über alle Sätze im Datensatz
for satz in emotionen["text"]:
  # teile den Satz anhand der Leerzeichen in einzelne Worte auf
  worte_in_satz = satz.split(" ")
  # iteriere über alle Worte im aktuellen Satz
  for wort in worte_in_satz:
    # wenn das Wort noch nicht in der Liste enthalten ist: füge es hinzu
    if wort not in alle_worte and len(wort) > 1:
      alle_worte.append(wort)

# gib die Länge der resultierenden Liste aus
print(f"Im Datensatz sind {len(alle_worte)} verschiedene Worte mit Länge > 1 enthalten.")

Im Datensatz sind 8951 verschiedene Worte mit Länge > 1 enthalten.


In [8]:
# Gegenprobe: auch der CountVectorizer liefert 8951 features zurück
count_vect = CountVectorizer()
count_vect.fit(emotionen['text'])
print(f"Anzahl der features im CountVectorizer: {len(count_vect.get_feature_names_out())}")

Anzahl der features im CountVectorizer: 8951


In [9]:
# wir können den Ansatz aber noch verfeinern, indem wir Worte, die selten
# vorkommen, ignorieren. Hier ignorieren wir alle Worte, die < 3 mal vorkommen:
count_vect = CountVectorizer(min_df=3)
count_vect.fit(emotionen['text'])
print(f"Anzahl der features im CountVectorizer: {len(count_vect.get_feature_names_out())}")

Anzahl der features im CountVectorizer: 2735


In [10]:
# Das sieht schon besser handhabbar aus! Wir transformieren also unseren
# Datensatz mit dem so trainierten CountVectorizer:
x_counts = count_vect.transform(x)

In [11]:
# dieser Datensatz hat 6000 Zeilen (Beobachtungen) und 2753 Spalten
# (features, Dimensionen)
x_counts.shape

(6000, 2735)

In [12]:
# wie sieht ein so transformierter Satz aus?
x_counts[0]

<1x2735 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [13]:
# so große Matrizen werden automatisch als "dünnbesetzte Matrizen" abgespeichert
# um Platz im Speicher zu sparen. Um ihren Inhalt anzuzeigen, müssen wir sie
# erst in eine normale Matrix transformieren
x_counts[0].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [14]:
# wie viele Einträge die nicht null sind hat der transformierte erste Satz?
x_counts[0].todense().sum()

10

In [15]:
# wie sieht der originale Satz aus?
# abzüglich der Worte mit Länge <2 sind 11 Worte in dem Satz enthalten.
# das heißt ein Wort ist herausgefallen, weil es im gesamten Datensatz weniger
# als 3x vorkommt.
emotionen["text"].iloc[0]

'i have to admit despite all my optimism i still feel a little uncertain'

## Ein Klassifikator für Emotionserkennung
Wir möchten nun mit dem Datensatz einen Klassifikator trainieren, der Emotionen in einem englischen Satz erkennen kann. Dafür folgen wir dem bekannten Training-Test Vorgehen aus [Kursteil 4](https://colab.research.google.com/drive/16wBhcXL4XLnqSSVw7Pefj0beTbL6-vKN#scrollTo=VENRtAug7f1D).

In [16]:
# wir teilen den Datensatz in einen Trainings- und einen Testdatensatz auf
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# wir transformieren die Beobachtungen im Trainings- und Testdatensatz jeweils
# mit dem vorher trainierten CountVektorizer. Achtung: hier ist wichtig, dass
# wir den CountVektorizer vorher auf dem gesamten Datensatz tainiert haben, da
# es durchaus Worte geben kann, die nach dem Aufteilen nur im Trainings, oder
# Testdatensatz vorkommen und die sonst nicht korrekt transformiert werden würden
x_train_counts = count_vect.transform(x_train)
x_test_counts = count_vect.transform(x_test)

## Übung 1

<ul class="outside">
<li><font color='blue'>Trainieren Sie einen <a href="https://scikit-learn.org/1.5/modules/generated/sklearn.ensemble.RandomForestClassifier.html">Entscheidungsbaum (random forest classifier)</a> mit den Trainingsdaten. Gehen Sie dafür vor wie in <a hreff="https://colab.research.google.com/drive/16wBhcXL4XLnqSSVw7Pefj0beTbL6-vKN#scrollTo=OsPfX83x2F2Y">Kurs 4</a> bzw. der <a href="https://colab.research.google.com/drive/1myBeee7BfPWCjxdqWL5gZk8buOz4gRpZ">Hausaufgabe</a> zu Kurs 4, Übung 1 (6).</font></li>
<li><font color='blue'>Sagen Sie die Emotionen für die Testdaten voraus.</font></li>
<li><font color='blue'>Messen Sie die Genauigkeit (accuracy) der Vorhersagen.</font></li>
<li><font color='blue'>Sagen Sie die Emotionen für die in der Codezelle unten gegebenen vier neuen Sätze voraus. <b>Hinweis</b>: vergessen Sie nicht, die Sätze vorher mit dem trainierten <tt>CountVectorizer</tt> zu transformieren!</font></li>
<li><font color='blue'>Überlegen Sie sich vier weitere Sätze und sagen Sie die Emotionen voraus. Entsprechen die Vorhersagen Ihren Erwartungen?</font></li>
</ul>


In [18]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(x_train_counts, y_train)
y_pred = classifier.predict(x_test_counts)

In [19]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.85

In [23]:
# 0: sadness
# 1: joy
# 2: love
# 3: anger
# 4: fear
# 5: surprise

saetze = [
    'i was feeling very angry, thinking of how my teacher treated me when i was in middle school',
    'when i come home and my cat greets me, i am feeling full of love for her. what a cute animal',
    'seeing the current state of the world i just want to crawl into bed and cry. its madness everywhere and i hate it',
    'yesterday i came home and my family had prepared a birthday party for me. i was so surprised, i didnt expect that'
]

saetze_counts = count_vect.transform(saetze)
saetze_pred = classifier.predict(saetze_counts)
saetze_pred

array([3, 1, 0, 5])

## Mehrklassen-Klassifikation (multiclass classification)
Bis jetzt hatten wir es beim überwachten Lernen für die Klassifikation nur mit binärer Klassifikation (überleben vs. nicht überleben, Brustkrebs vs. kein Brustkrebs) zu tun. Im Fall der Emotionen haben wir aber mehrere Klassen (Emotionen). Hier sprechen wir von der "Mehrklassen-Klassifikation" (multiclass classification).  

Außerdem kann es vorkommen, dass wir einer Beobachtung mehr als eine Klasse zuweisen möchten (behandeln wir aber nicht in dieser Lehrveranstaltung). Dann sprechen wir von "Mehrlabel Klassifikation" (multilabel classification).


<div>
<img src="https://drive.google.com/uc?id=1czgo3Mo4uWesjYWYXp_BDDRolfms0bOO" width="800"/>
</div>

Die Genauigkeit von einem Mehrklassen-Klassifikator zu messen ist einfach. Wir können einfach die Fälle zählen, in denen der Klassifikator nicht richtig liegt.  

Im Fall von anderen [Performancemaßen](https://janalasser.at/lectures/MC_KI/VO2_2_performance/#/2/0/1) wie dem positivem Vorhersagewert (precision) und der Sensitivität (recall) ist das Vorgehen nicht ganz so klar - es gibt nämlich für jede der Klassen einen eigenen positiven Vorhersagewert und eine eigene Sensitivität.

In [36]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(verbose=True, random_state=0)
classifier.fit(x_train_counts, y_train)
y_pred = classifier.predict(x_test_counts)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    1.7s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [37]:
# für welche der 6 Emotionen funktioniert die vorhersage am besten?
# 0: sadness
# 1: joy
# 2: love
# 3: anger
# 4: fear
# 5: surprise

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.78      0.77       186
           1       0.82      0.72      0.77       206
           2       0.84      0.95      0.89       190
           3       0.90      0.83      0.86       206
           4       0.93      0.83      0.87       207
           5       0.86      0.98      0.91       205

    accuracy                           0.85      1200
   macro avg       0.85      0.85      0.85      1200
weighted avg       0.85      0.85      0.85      1200



## Übung 2

<ul class="outside">
<li><font color='blue'>Jede:r sucht sich eine Emotion aus und schreibt 10 Sätze, die die Emotion ausrücken sollen in dieses <a href="https://docs.google.com/document/d/1-U_YxRfU0q10_rLh_gFrwUqaRhBqv9znYUwQfqGdoo4/edit?usp=sharing">Google Doc</a>.</font></li>
<li><font color='blue'>Kopieren Sie alle Sätze in die Liste <tt>x_test_neu</tt> in der untenstehenden Code Zelle. Achten Sie auf die Reihenfolge: zuerst alle Sätze zu "sadness", dann alle zu "joy" etc. </font></li>
<li><font color='blue'>Sagen Sie für alle neuen Sätze die enthaltene Emotion voraus. <b>Hinweis</b>: vergessen Sie nicht, die Sätze zuerst mit dem trainierten <tt>CountVectorizer</tt> zu transformieren!</font></li>
<li><font color='blue'>Messen Sie die Performance des Klassifikators auf den neuen Sätzen indem sie einen <tt>classification_report</tt> erstellen. Wie gut performt der Klassifikator auf diesem komplett neuen Datensatz, also "out of sample"?</font></li>
</ul>







In [38]:
x_test_neu = [
    "I regret very much to be in Graz in Winter.", # sadness
    "My grandmother was a funny person, but she passed away long ago.", # sadness
    "I could not go to Gabon, because I was denied a visa.", # sadness
    "Sometimes I sit near a lake and read Solschenizyn.", # sadness
    "We must expect that AGI will cause many conflicts with China.", # sadness
    "Politics is a dirty business, in America as well as in Israel.", # sadness
    "It took me a long time to write a paper about extremism.", # sadness
    "All experts about climate change are losing hope.", # sadness
    "There is not much the government can do to stimulate the economy due to fiscal constraints.", # sadness
    "February will be boring without lectures at the university.", # sadness
    "My dog is happy when he see me.", # joy
    "I passed my exam.", # joy
    "Paul is a friendly cat.", # joy
    "Sun is my favorite type of weather.", # joy
    "I'm thrilled to hear that.", # joy
    "That's wonderful news!", # joy
    "I can't believe how lucky I am.", # joy
    "I just got a promotion at work.", # joy
    "I finally finished that book I've been wanting to read for months.", # joy
    "My favorite band is coming to town.", # joy
    "I love you.", # love
    "Today I received a gift from my girlfriend.",# love
    "When two people like each other, that’s called love.",# love
    "A girl I know is pretty funny, I think I like her.",# love
    "When I’m separated from you, that makes me sad.",# love
    "The boy, who I really like, texted me.",# love
    "My favorite season is summer, I love it.",# love
    "I love Spaghetti.",# love
    "I have to go to uni today, because a girl I like asked me to.",# love
    "Puppies are so cute, I love them.",# love
    "I lost my wallet.", # anger
    "Today I got a 5 in my final exam.", # anger
    "I was so angry that I slammed the door behind me.", # anger
    "When she lied to me, I could feel the anger inside me.", # anger
    "The employee had to work overtime.", # anger
    "He cheated on me on our anniversary.", # anger
    "I found the man who killed my cat on purpose.", # anger
    "With one click, I lost everything I had.", # anger
    "I hate going on family events.", # anger
    "My enemy is sitting across in front of me on the bus.", # anger
    "I heard loud footsteps behind me and did not even dare turn around to see who it was.", # fear
    "I was standing at the edge of a cliff when it hit me, that I could actually fall down.", # fear
    "It felt like someone was watching me from the window.", # fear
    "A loud banging noise on my door woke me up in the middle of the night.", # fear
    "I was standing in a creepy basement when all of a sudden the lights went off.", # fear
    "A man was chasing me and I was starting to get out of breath.", # fear
    "The neighbor's aggressive dog escaped his house and started to run at me.", # fear
    "I heard someone saying my name but when I looked around, there was nobody around.", # fear
    "I was on a boat in the ocean when a huge wave got me and pushed me out further into the open water.", # fear
    "The teacher saw me sleeping in class and decided to punish me by letting me take a surprise math test.", # fear
    "This morning I forgot my phone. I was in for a nasty surprise when I arrived at work later that day.", # surprise
    "Wow, I just won in the lottery!", # surprise
    "When I found out my partner had another girlfriend, I was baffled.", # surprise
    "Yesterday I came home and my friends had organised a birthday party for me. I really didn’t expect that!", # surprise
    "I wanted to cook curry for lunch but I didn’t have any spices at home. That was unexpected.", # surprise
    "This morning, I left my phone at home. I was in for a rude shock when I got to work later in the day.", # surprise
    "Incredible, I've just hit the jackpot in the lottery!", # surprise
    "I was completely bewildered when I discovered my partner had another girlfriend.", # surprise
    "When I returned home yesterday, my friends had thrown a surprise birthday party for me. I really didn't see that coming!", # surprise
    "I planned to make curry for lunch, but I didn't have any spices at home. That caught me off guard." # surprise
]

y_test_neu = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
    2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
    3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
    4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5
]


In [39]:
x_test_neu_counts = count_vect.transform(x_test_neu)

In [40]:
y_pred_neu = classifier.predict(x_test_neu_counts)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [41]:
print(classification_report(y_test_neu, y_pred_neu))

              precision    recall  f1-score   support

           0       0.18      0.60      0.28        10
           1       0.26      0.50      0.34        10
           2       0.00      0.00      0.00        10
           3       0.50      0.20      0.29        10
           4       1.00      0.20      0.33        10
           5       0.00      0.00      0.00        10

    accuracy                           0.25        60
   macro avg       0.32      0.25      0.21        60
weighted avg       0.32      0.25      0.21        60



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Weiterführende Materialien
* **Embeddings mit vortrainierten Modellen**: [NLTK tutorial](https://www.nltk.org/howto/gensim.html)
* **Micro- und Macro F1 Score explained**: [Blog post](https://iamirmasoud.com/2022/06/19/understanding-micro-macro-and-weighted-averages-for-scikit-learn-metrics-in-multi-class-classification-with-example/).
* **How LLMs work**: [Video](https://www.youtube.com/watch?v=wjZofJX0v4M) von 3Blue1Brown (enthält auch einen Kurzen Teil über Embeddings).
* **Emotionserkennung und der AI Act**: [Einordnung](https://www.youtube.com/watch?v=wjZofJX0v4M) von Systemen zur Emotionserkennung im AI Act.

## Quelle und Lizenz

Das vorliegende Notebook wurde von Jana Lasser für den Kurs B "technische Aspekte" des Microcredentials "KI und Gesellschaft" der Universität Graz erstellt.

Das Notebook kann unter den Bedingungen der Lizenz [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0) verwendet, modifiziert und weiterverbreitet werden.
